# Caso practico AI Engineer Tech lead
## QPH: Evaluacion
### AUTOR: Isaac Reyes
#### Notebook jupyter ipynb PYTHON
##### CODIGO: para subir a repo

In [12]:
#instalar librerias
!pip install pandas openpyxl prophet

In [8]:
#subir xlsx:
import os

try:
    #Si estamos en Colab, usamos el widget de subida
    from google.colab import files
    print("Arrastrar y suelta tu archivo Excel de reglas aquí:")
    uploaded = files.upload()
    #Tomo el primer archivo que subo
    excel_filename = next(iter(uploaded.keys()))
    excel_path = f"/content/{excel_filename}"
    print(f"Archivo cargado: {excel_path}")
except Exception:
    #Si no es Colab, tomo la ruta local
    excel_path = "/mnt/data/Ventas e inventarios juguetes.xlsx"
    print(f"No se detectó Colab. Usando ruta local: {excel_path}")

#Ahora excel_path apunta al archivo que usarás en la siguiente celda


Arrastrar y suelta tu archivo Excel de reglas aquí:


Saving Ventas e inventarios juguetes.xlsx to Ventas e inventarios juguetes.xlsx
Archivo cargado: /content/Ventas e inventarios juguetes.xlsx


In [14]:
#traigo librerias importo
import os
import pandas as pd
from prophet import Prophet
from google.colab import files
from IPython.display import display

In [17]:
def load_data(filepath):
    df_2023 = pd.read_excel(filepath, sheet_name='Ventas 2023', engine='openpyxl')
    df_2024 = pd.read_excel(filepath, sheet_name='Ventas Reales 2024', engine='openpyxl')
    df_inv  = pd.read_excel(filepath, sheet_name='Inventarios', engine='openpyxl')
    return df_2023, df_2024, df_inv

df23, df24, df_inv = load_data(excel_path)
print("Datos cargados")


Datos cargados


In [21]:
#Preparo series de tiempo unificadas
def prepare_time_series(df_2023, df_2024):
    #Mapeo manual de meses en español
    month_map={
        "Enero": 1, "Febrero": 2, "Marzo": 3, "Abril": 4,
        "Mayo": 5, "Junio": 6, "Julio": 7, "Agosto": 8,
        "Septiembre": 9, "Octubre": 10, "Noviembre": 11, "Diciembre": 12
    }
    #Procesar 2023 O MELT
    df23 = df_2023.melt(id_vars=['Producto'], var_name='Mes', value_name='Ventas')
    df23['month'] = df23['Mes'].map(month_map)
    df23['year']  = 2023
    df23['ds'] = pd.to_datetime(
        df23[['year','month']].assign(day=1)
    )
    #Procesar 2024
    varcols= df_2024.columns[1:]
    df24=df_2024.melt(id_vars=['Producto'], value_vars=varcols, var_name='Mes', value_name='Ventas')
    df24['month'] = df24['Mes'].map(month_map)
    df24['year']  = 2024
    df24['ds']    = pd.to_datetime(
        df24[['year','month']].assign(day=1)
    )
    #Unir ambas series
    ts =pd.concat([
        df23[['ds','Producto','Ventas']],
        df24[['ds','Producto','Ventas']]
    ], ignore_index=True)
    return ts.dropna().sort_values(['Producto','ds'])

#Pruebo y veo:
ts = prepare_time_series(df23, df24)
print("Series de tiempo preparadas:", ts.shape)

Series de tiempo preparadas: (105, 3)


In [22]:
#generar forecast Q4 2024
def forecast_q4(ts, horizon=3):
    frames = []
    for prod in ts['Producto'].unique():
        dfp = ts[ts['Producto']==prod][['ds','Ventas']].rename(columns={'Ventas':'y'})
        m = Prophet(yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False)
        m.fit(dfp)
        future = m.make_future_dataframe(periods=horizon, freq='M')
        pred = m.predict(future)
        q4 = pred[['ds','yhat']].tail(horizon).assign(Producto=prod)
        frames.append(q4)
    return pd.concat(frames, ignore_index=True)

q4_forecast = forecast_q4(ts)
print("Forecast Q4 generado")

INFO:prophet:n_changepoints greater than number of observations. Using 15.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp0x61syiv/x1xe_c53.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp0x61syiv/2izn36yn.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=14006', 'data', 'file=/tmp/tmp0x61syiv/x1xe_c53.json', 'init=/tmp/tmp0x61syiv/2izn36yn.json', 'output', 'file=/tmp/tmp0x61syiv/prophet_modelfq__59cm/prophet_model-20250527155530.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
15:55:30 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
15:55:44 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
/usr/local/lib/python3.11/dist-packages/prophet/forecaster.py:1854: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.

Forecast Q4 generado


/usr/local/lib/python3.11/dist-packages/prophet/forecaster.py:1854: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(


In [25]:
#analisis inv vs. demanda
def analyze_inventory(forecast_df, df_2024, df_inv):
    #1)Ventas YTD en sp 2024
    ytd = (
        df_2024
        .set_index('Producto')
        .iloc[:, 1:10]
        .sum(axis=1)
        .rename('YTD_2024')
    )

    #2)Proyec Q4 2024
    proj = (
        forecast_df
        .groupby('Producto')['yhat']
        .sum()
        .rename('Q4_2024_proj')
    )

    #3. est anual 2024
    est = pd.concat([ytd, proj], axis=1).assign(
        Est_2024=lambda df: df['YTD_2024'] + df['Q4_2024_proj']
    )

    #4.Inventario total (Actual+Entregas Q4)
    inv = df_inv.set_index('Producto')
    inv_tot = (
        inv['Inventario Actual']
        + inv[['Entrega Programada Octubre',
               'Entrega Programada Noviembre',
               'Entrega Programada Diciembre']].sum(axis=1)
    )

    #5)Unión y clasificación
    df = (
        est
        .join(inv_tot.rename('Total_Inv'))
        .reset_index()
    )
    df['Diff']   = df['Total_Inv'] - df['Est_2024']
    df['Status'] = df['Diff'].apply(lambda x: 'Exceso inv.' if x > 0 else 'Riesgo stockout')
    return df

#Ejecutar y compruebo
inventory_df = analyze_inventory(q4_forecast, df24, df_inv)
print("Análisis de inventario completado")


Análisis de inventario completado


In [26]:
print("Análisis de Inventario 2024:")
from IPython.display import display
display(inventory_df)
#aqui muestro el inv analizado

Análisis de Inventario 2024:


,Producto,YTD_2024,Q4_2024_proj,Est_2024,Total_Inv,Diff,Status
0,Mandy Princesa,10110,4231.483214,14341.483214,1000,-13341.483214,Riesgo stockout
1,Mandy Dentista,5810,1545.580622,7355.580622,1000,-6355.580622,Riesgo stockout
2,Mandy Veterinaria,8100,2243.313952,10343.313952,490,-9853.313952,Riesgo stockout
3,Mandy Sirena,5140,-1320.158005,3819.841995,5120,1300.158005,Exceso inv.
4,Mandy Doctora,6150,1464.456528,7614.456528,630,-6984.456528,Riesgo stockout


In [27]:
os.makedirs('/content/output', exist_ok=True)
inventory_df.to_csv('/content/output/inventory_analysis_2024.csv', index=False)
print("Inventory analysis CSV guardado en /content/output/inventory_analysis_2024.csv")

Inventory analysis CSV guardado en /content/output/inventory_analysis_2024.csv


In [28]:
print("Productos y Estado:")
print(inventory_df[['Producto','Status','Diff']])
#resumen

Productos y Estado:
            Producto           Status          Diff
0     Mandy Princesa  Riesgo stockout -13341.483214
1     Mandy Dentista  Riesgo stockout  -6355.580622
2  Mandy Veterinaria  Riesgo stockout  -9853.313952
3       Mandy Sirena      Exceso inv.   1300.158005
4      Mandy Doctora  Riesgo stockout  -6984.456528


In [29]:
print("Pipeline completado: ya tienes el forecast y el análisis de inventario listos.")
#mensajeria rapida

Pipeline completado: ya tienes el forecast y el análisis de inventario listos.


In [30]:
#Resumen completo del caso práctico
print("Resumen del Caso Práctico – IA: Ventas e Inventarios Juguetes\n")
print("1)Proyección de Ventas Oct–Dic 2024 (Prophet):")
for prod in q4_forecast['Producto'].unique():
    sub = q4_forecast[q4_forecast['Producto']==prod]
    oct_v = sub.iloc[0]['yhat']
    nov_v = sub.iloc[1]['yhat']
    dic_v = sub.iloc[2]['yhat']
    print(f"{prod}: Octubre={oct_v:,.0f}, Noviembre={nov_v:,.0f}, Diciembre={dic_v:,.0f}")
print()

print("2)Ventas Anuales Estimadas 2024 vs. Inventario Total (Actual + Entregas Q4):")
for _, row in inventory_df.iterrows():
    print(f"{row['Producto']}: Est. Anual={row['Est_2024']:,.0f}  |  Inventario Total={row['Total_Inv']:,.0f}  |  Diff={row['Diff']:,.0f}  => {row['Status']}")
print()

print("3 Recomendaciones:")
# Exceso
exceso = inventory_df[inventory_df['Status']=='Exceso inv.']['Producto'].tolist()
if exceso:
    print("a)Exceso de Inventario en:", ", ".join(exceso))
    print("Promociones flash y bundles (compra 2 lleva 3).")
    print("Incentivos y rebates para detallistas")
    print("Descuentos especiales antes del 01-01-2025.\n")
# Riesgo stockout
stockout = inventory_df[inventory_df['Status']=='Riesgo stockout']['Producto'].tolist()
if stockout:
    print("b)Riesgo de Quiebre de Stock en:", ", ".join(stockout))
    print("Emitir Órdenes de Compra INMEDIATAS (lead time = 60 días).")
    print("Confirmar llegada antes del 25 de diciembre.")
    print("Monitoreo semanal de inventario vs. ventas reales/proyectadas.\n")

Resumen del Caso Práctico – IA: Ventas e Inventarios Juguetes

1)Proyección de Ventas Oct–Dic 2024 (Prophet):
Mandy Dentista: Octubre=363, Noviembre=638, Diciembre=544
Mandy Doctora: Octubre=670, Noviembre=544, Diciembre=250
Mandy Princesa: Octubre=1,128, Noviembre=1,172, Diciembre=1,931
Mandy Sirena: Octubre=-556, Noviembre=3,775, Diciembre=-4,540
Mandy Veterinaria: Octubre=579, Noviembre=875, Diciembre=790

2)Ventas Anuales Estimadas 2024 vs. Inventario Total (Actual + Entregas Q4):
Mandy Princesa: Est. Anual=14,341  |  Inventario Total=1,000  |  Diff=-13,341  => Riesgo stockout
Mandy Dentista: Est. Anual=7,356  |  Inventario Total=1,000  |  Diff=-6,356  => Riesgo stockout
Mandy Veterinaria: Est. Anual=10,343  |  Inventario Total=490  |  Diff=-9,853  => Riesgo stockout
Mandy Sirena: Est. Anual=3,820  |  Inventario Total=5,120  |  Diff=1,300  => Exceso inv.
Mandy Doctora: Est. Anual=7,614  |  Inventario Total=630  |  Diff=-6,984  => Riesgo stockout

3 Recomendaciones:
a)Exceso de Inve